# foreachRDD Demo

`dstream.foreachRDD` is a powerful primitive that allows data to be sent out to external systems. However, it is important to understand how to use this primitive correctly and efficiently. Some of the common mistakes to avoid are as follows.

Often writing data to external system requires creating a connection object (e.g. TCP connection to a remote server) and using it to send data to a remote system. For this purpose, a developer may inadvertently try creating a connection object at the Spark driver, and then try to use it in a Spark worker to save records in the RDDs. For example (in Scala),
```python
def sendRecord(rdd):
    connection = createNewConnection()  # executed at the driver
    rdd.foreach(lambda record: connection.send(record))
    connection.close()

dstream.foreachRDD(sendRecord)
```
This is incorrect as this requires the connection object to be serialized and sent from the driver to the worker. Such connection objects are rarely transferable across machines. This error may manifest as serialization errors (connection object not serializable), initialization errors (connection object needs to be initialized at the workers), etc. The correct solution is to create the connection object at the worker.

However, this can lead to another common mistake - creating a new connection for every record. For example,
```python
def sendRecord(record):
    connection = createNewConnection()
    connection.send(record)
    connection.close()

dstream.foreachRDD(lambda rdd: rdd.foreach(sendRecord))
```
Typically, creating a connection object has time and resource overheads. Therefore, creating and destroying a connection object for each record can incur unnecessarily high overheads and can significantly reduce the overall throughput of the system. A better solution is to use `rdd.foreachPartition` - create a single connection object and send all the records in a RDD partition using that connection.
```python
def sendPartition(iter):
    connection = createNewConnection()
    for record in iter:
        connection.send(record)
    connection.close()

dstream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
```
This amortizes the connection creation overheads over many records.

Finally, this can be further optimized by reusing connection objects across multiple RDDs/batches. One can maintain a static pool of connection objects than can be reused as RDDs of multiple batches are pushed to the external system, thus further reducing the overheads.
```python
def sendPartition(iter):
    # ConnectionPool is a static, lazily initialized pool of connections
    connection = ConnectionPool.getConnection()
    for record in iter:
        connection.send(record)
    # return to the pool for future reuse
    ConnectionPool.returnConnection(connection)

dstream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
```
Note that the connections in the pool should be lazily created on demand and timed out if not used for a while. This achieves the most efficient sending of data to external systems.

**Other points to remember:**

* DStreams are executed lazily by the output operations, just like RDDs are lazily executed by RDD actions. Specifically, RDD actions inside the DStream output operations force the processing of the received data. Hence, if your application does not have any output operation, or has output operations like `dstream.foreachRDD()` without any RDD action inside them, then nothing will get executed. The system will simply receive the data and discard it.
* By default, output operations are executed one-at-a-time. And they are executed in the order they are defined in the application.


### Demo

In [1]:
import findspark
# TODO: your path will likely not have 'matthew' in it. Change it to reflect your path.
findspark.init('/home/matthew/spark-2.1.0-bin-hadoop2.7')

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [3]:
sc = SparkContext(master="local[2]", appName="SparkforEachRDDapp")
ssc = StreamingContext(sc, 10)

In [4]:
myFile = ssc.textFileStream("data")
wordspair = myFile.flatMap(lambda row: row.split(" ")).map(lambda x: (x, 1)).reduceByKey(lambda x,y : x + y)
oldwordcount = wordspair.reduceByKey(lambda x,y : x + y)

In [5]:
def rdd_print(rdd):
    a = rdd.collect()
    print(a)

In [6]:
oldwordcount.foreachRDD(rdd_print)

In [7]:
ssc.start()

[]
[]
[('', 196), ('step', 1), ('No!', 3), ('was', 20), ('sunny.', 1), ('quiet', 1), ('Just', 1), ('head', 5), ('WALK', 3), ('bangs', 1), ('hop?', 1), ('frown,', 1), ('kite', 3), ('in', 117), ('start', 1), ('band', 2), ('books!', 2), ('Ying.', 1), ('yell.', 1), ('as', 11), ('why', 1), ('Whos,', 4), ('do?"', 1), ('fall.', 1), ('star.', 1), ('right', 4), ("That's", 5), ('Will', 6), ('eyes!', 1), ('kites,"', 1), ('whose', 3), ('three...', 1), ('Things', 9), ('work', 1), ('cat', 4), ('Without', 1), ('dark.', 5), ('home!', 1), ('snarled', 1), ('done,', 1), ('too.', 2), ('Two', 6), ('"Giddap!"', 1), ('her', 8), ('poodle', 3), ('Down', 2), ('Christmas!', 2), ('new', 8), ('Staring', 1), ('shine.', 1), ('fear', 1), ('quarter', 1), ('cup.', 1), ('TALL', 1), ('glad,', 1), ('"What', 1), ('got', 5), ('took', 8), ('books,', 1), ('run.', 1), ('Hat.', 4), ('Gump.', 1), ('Leaving', 1), ('around', 1), ('games', 1), ('pet?', 1), ('hose', 2), ('RED', 4), ('Goo-Goose,', 1), ('fooled', 1), ('jump', 1), ('ne

[]


In [8]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

[]
[]


## References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#design-patterns-for-using-foreachrdd